### 加载模型

In [1]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="shmily_006/Qw3",
    base_url="http://localhost:11434/v1",
    api_key="EMPTY",
)

### 用大模型做文本分类

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
从以下段落中提取所需的信息。

仅提取 'Classification' 函数中提到的属性。

内容：
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="情绪标签")
    aggressiveness: int = Field(
        description="文本在 1 到 10 范围内的情绪激进程度"
    )
    language: str = Field(description="语言")


# 大模型结构化输出
structured_llm = llm.with_structured_output(Classification)

In [3]:
inp = "我非常高兴认识了你！我想我们会成为真正的好朋友！"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response

Classification(sentiment='positive', aggressiveness=0, language='Chinese')

In [6]:
inp = "我对你很生气！！"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response.model_dump()

{'sentiment': 'negative', 'aggressiveness': 2, 'language': 'Chinese'}

### 更精细化的文本分类

In [7]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["开心", "中性", "伤心"])
    aggressiveness: int = Field(
        ...,
        description="描述语句的情绪激进程度，数字越高，越激进",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["西班牙语", "英语", "法语", "德语", "中文"]
    )

In [8]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
从以下段落中提取所需的信息。

仅提取 'Classification' 函数中提到的属性。

内容：
{input}
"""
)

llm = llm = ChatOpenAI(
    model="shmily_006/Qw3",
    base_url="http://localhost:11434/v1",
    api_key="EMPTY",
).with_structured_output(Classification)

In [9]:
inp = "我非常高兴认识了你！我想我们会成为非常好的朋友！"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='开心', aggressiveness=1, language='中文')

In [10]:
inp = "我对你很生气！"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='伤心', aggressiveness=2, language='中文')

In [11]:
inp = "这里的天气还不错"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='中性', aggressiveness=1, language='中文')